# Обучение регрессионных задач по проекту ML 2024

## California housing

Установка необходимых библиотек.

In [5]:
#pip install lightgbm
!pip install catboost
#pip install xgboost
#pip install torch
!pip install rarfile
# !pip install xgboost
!pip install lightgbm

Импорт используемых библиотек.

In [6]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn import linear_model
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from scipy.io import arff
from os import listdir, getcwd, chdir, mkdir
import xgboost
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge
import numpy as np
from sklearn.utils import resample
from scipy import stats
from scipy.stats import t
from catboost import CatBoostRegressor
import math
import warnings
warnings.filterwarnings("ignore")
from collections import OrderedDict
from typing import Any, Dict, List, Optional, Union
try:
    import sklearn.tree as sklearn_tree
except ImportError:
    sklearn_tree = None

import torch
import torch.nn as nn
from torch import Tensor
try:
    from tqdm import tqdm
except ImportError:
    tqdm = None

import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
import lightgbm as lg
import rarfile


Для простоты я написал несколько функций, их импортироуем тут.

In [7]:
def boostraping(model, x_test,
                y_test, n_iterations = 20):
  """forms list of rmse scores

  Keyword arguments:
  model -- fitted model
  x_test -- features of test data
  y_test -- labels of test data
  n_iterations -- number of boostraping procedures (default 20)
  """
  accuracy = []
  for i in range(n_iterations):
      X_bs, y_bs = resample(x_test, y_test, replace=True)
      # make predictions
      y_hat = model.predict(X_bs)
      # evaluate model
      score = accuracy_score(y_bs, y_hat)
      accuracy.append(score)
  return accuracy

def project_train_simple(x_train, x_test, y_train, y_test):
  """Train project models and generate rmse scores and fitted models

  Keyword arguments:
  x_train -- features of train data
  y_train -- labels of train data
  x_test -- features of test data
  y_test -- labels of test data
  """
  models = [
            CatBoostClassifier(), LogisticRegression(),
            RandomForestClassifier(), XGBClassifier(), LGBMClassifier()
            ]

  test_score = []
  train_score = []
  model = []
  best_parameters = []
  for i, k in enumerate(models):
    print('Training '+str(k.__class__.__name__))
    # Step 1: Fit model
    k.fit(x_train, y_train)
    model.append(k.__class__.__name__)
    # Step 2: Calculate rmse scores
    train_score.append(accuracy_score(y_train, k.predict(x_train)))
    test_score.append(accuracy_score(y_test, k.predict(x_test)))
    best_parameters.append(k)
  data = {'model name': model, 'train accuracy': train_score, 'test accuracy': test_score}
  df = pd.DataFrame(data)
  return df, best_parameters

def one_sample_t_test(sample, population_mean, alpha=0.05, tail="two"):
    """Make t-test and check
    H0: mean value of sample == population_mean

    Keyword arguments:
    sample -- list of means of fetches
    population_mean -- mean of distribution
    alpha -- the threshold value for rejecting a H0 (default 0.05) 0.01 for 99%
    tail -- method of the p-value calculation (default = "two", "left", "right")
    """
    # Step 1: Calculate T-score
    sample_mean = np.mean(sample)
    sample_std = np.std(sample, ddof=1)
    sample_size = len(sample)

    t_score = (sample_mean - population_mean) / \
        (sample_std / np.sqrt(sample_size))

    # Step 2: Determine degrees of freedom
    df = sample_size - 1

    # Step 3: Identify the appropriate t-distribution
    # No need to explicitly specify degrees of freedom for one-sample t-test in scipy.stats.t

    # Step 4: Find the p-value
    if tail == "two":
        p_value = t.sf(np.abs(t_score), df) * 2  # for two-tailed test
    elif tail == "left":
        p_value = t.sf(t_score, df)  # for left-tailed test
    elif tail == "right":
        p_value = t.sf(-t_score, df)  # for right-tailed test
    else:
        raise ValueError(
            "Invalid tail argument. Use 'two', 'left', or 'right'.")

    # Step 5: Interpret the p-value
    print("P-value:", p_value)

    if p_value < alpha:
        print(
            "Reject the null hypothesis. There is a statistically significant difference.")
    else:
        print("Fail to reject the null hypothesis. There is no statistically significant difference.")
    return p_value


def project_train(x_train, x_test, y_train, y_test):
  """Train project models on gridsearchCV and generate rmse scores and fitted models

  Keyword arguments:
  x_train -- features of train data
  y_train -- labels of train data
  x_test -- features of test data
  y_test -- labels of test data
  """
  models = [
            CatBoostClassifier(), LogisticRegression(),
            RandomForestClassifier(), XGBClassifier(), LGBMClassifier()
            ]

  grid = [
          {'depth': [4, 6, 8],
          'learning_rate': [0.01, 0.05, 0.1],
          'iterations': [100, 200]},

          {'C': [0.1, 1, 10],
          'penalty': ['none', 'l2']},

          {'n_estimators': [100, 200, 300],
          'max_depth': [None, 10, 20]},

          {'max_depth': [3, 5, 7],
          'learning_rate': [0.01, 0.05, 0.1],
          'n_estimators': [100, 200]},

          {'max_depth': [3, 5, 7],
          'learning_rate': [0.01, 0.05, 0.1],
          'n_estimators': [100, 200]}
          ]

  test_score = []
  train_score = []
  model = []
  best_parameters = []
  for i, k in enumerate(models):
    print('Training '+str(k.__class__.__name__))
    clf = GridSearchCV(k, grid[i])
    # Step 1: Fit model on gridsearchCV
    best_estimator = clf.fit(x_train, y_train)
    best_parameters.append(best_estimator)
    model.append(k.__class__.__name__)
    # Step 2: Calculate rmse scores
    train_score.append(accuracy_score(y_train, best_estimator.predict(x_train)))
    test_score.append(accuracy_score(y_test, best_estimator.predict(x_test)))

  data = {'model name': model, 'train accuracy': train_score, 'test accuracy': test_score}
  df = pd.DataFrame(data)
  return df, best_parameters

def _check_bins(bins: List[Tensor]) -> None:
    if not bins:
        raise ValueError('The list of bins must not be empty')
    for i, feature_bins in enumerate(bins):
        if not isinstance(feature_bins, Tensor):
            raise ValueError(
                'bins must be a list of PyTorch tensors. '
                f'However, for {i=}: {type(bins[i])=}'
            )
        if feature_bins.ndim != 1:
            raise ValueError(
                'Each item of the bin list must have exactly one dimension.'
                f' However, for {i=}: {bins[i].ndim=}'
            )
        if len(feature_bins) < 2:
            raise ValueError(
                'All features must have at least two bin edges.'
                f' However, for {i=}: {len(bins[i])=}'
            )
        if not feature_bins.isfinite().all():
            raise ValueError(
                'Bin edges must not contain nan/inf/-inf.'
                f' However, this is not true for the {i}-th feature'
            )
        if (feature_bins[:-1] >= feature_bins[1:]).any():
            raise ValueError(
                'Bin edges must be sorted.'
                f' However, the for the {i}-th feature, the bin edges are not sorted'
            )
        if len(feature_bins) == 2:
            warnings.warn(
                f'The {i}-th feature has just two bin edges, which means only one bin.'
                ' Strictly speaking, using a single bin for the'
                ' piecewise-linear encoding should not break anything,'
                ' but it is the same as using sklearn.preprocessing.MinMaxScaler'
            )


def compute_bins(
    X: torch.Tensor,
    n_bins: int = 48,
    *,
    tree_kwargs: Optional[Dict[str, Any]] = None,
    y: Optional[Tensor] = None,
    regression: Optional[bool] = None,
    verbose: bool = False,
) -> List[Tensor]:
    """Compute bin edges for `PiecewiseLinearEmbeddings`.

    **Usage**

    Computing the quantile-based bins (Section 3.2.1 in the paper):

    >>> X_train = torch.randn(10000, 2)
    >>> bins = compute_bins(X_train)

    Computing the tree-based bins (Section 3.2.2 in the paper):

    >>> X_train = torch.randn(10000, 2)
    >>> y_train = torch.randn(len(X_train))
    >>> bins = compute_bins(
    ...     X_train,
    ...     y=y_train,
    ...     regression=True,
    ...     tree_kwargs={'min_samples_leaf': 64, 'min_impurity_decrease': 1e-4},
    ... )

    Args:
        X: the training features.
        n_bins: the number of bins.
        tree_kwargs: keyword arguments for `sklearn.tree.DecisionTreeRegressor`
            (if ``regression`` is `True`) or `sklearn.tree.DecisionTreeClassifier`
            (if ``regression`` is `False`).
            NOTE: requires ``scikit-learn>=1.0,>2`` to be installed.
        y: the training labels (must be provided if ``tree`` is not None).
        regression: whether the labels are regression labels
            (must be provided if ``tree`` is not None).
        verbose: if True and ``tree_kwargs`` is not None, than ``tqdm``
            (must be installed) will report the progress while fitting trees.
    Returns:
        A list of bin edges for all features. For one feature:

        - the maximum possible number of bin edges is ``n_bins + 1``.
        - the minumum possible number of bin edges is ``1``.
    """
    if not isinstance(X, Tensor):
        raise ValueError(f'X must be a PyTorch tensor, however: {type(X)=}')
    if X.ndim != 2:
        raise ValueError(f'X must have exactly two dimensions, however: {X.ndim=}')
    if X.shape[0] < 2:
        raise ValueError(f'X must have at least two rows, however: {X.shape[0]=}')
    if X.shape[1] < 1:
        raise ValueError(f'X must have at least one column, however: {X.shape[1]=}')
    if not X.isfinite().all():
        raise ValueError('X must not contain nan/inf/-inf.')
    if (X == X[0]).all(dim=0).any():
        raise ValueError(
            'All columns of X must have at least two distinct values.'
            ' However, X contains columns with just one distinct value.'
        )
    if n_bins <= 1 or n_bins >= len(X):
        raise ValueError(
            'n_bins must be more than 1, but less than len(X), however:'
            f' {n_bins=}, {len(X)=}'
        )

    if tree_kwargs is None:
        if y is not None or regression is not None or verbose:
            raise ValueError(
                'If tree_kwargs is None, then y must be None, regression must be None'
                ' and verbose must be False'
            )

        # NOTE[DIFF]
        # The original implementation in the official paper repository has an
        # unintentional divergence from what is written in the paper.
        # This package implements the algorithm described in the paper,
        # and it is recommended for future work
        # (this may affect the optimal number of bins
        #  reported in the official repository).
        #
        # Additional notes:
        # - this is the line where the divergence happens:
        #   (the thing is that limiting the number of quantiles by the number of
        #   distinct values is NOT the same as removing identical quantiles
        #   after computing them)
        #   https://github.com/yandex-research/tabular-dl-num-embeddings/blob/c1d9eb63c0685b51d7e1bc081cdce6ffdb8886a8/bin/train4.py#L612C30-L612C30
        # - for the tree-based bins, there is NO such divergence;
        bins = [
            q.unique()
            for q in torch.quantile(
                X, torch.linspace(0.0, 1.0, n_bins + 1).to(X), dim=0
            ).T
        ]
        _check_bins(bins)
        return bins
    else:
        if sklearn_tree is None:
            raise RuntimeError(
                'The scikit-learn package is missing.'
                ' See README.md for installation instructions'
            )
        if y is None or regression is None:
            raise ValueError(
                'If tree_kwargs is not None, then y and regression must not be None'
            )
        if y.ndim != 1:
            raise ValueError(f'y must have exactly one dimension, however: {y.ndim=}')
        if len(y) != len(X):
            raise ValueError(
                f'len(y) must be equal to len(X), however: {len(y)=}, {len(X)=}'
            )
        if y is None or regression is None:
            raise ValueError(
                'If tree_kwargs is not None, then y and regression must not be None'
            )
        if 'max_leaf_nodes' in tree_kwargs:
            raise ValueError(
                'tree_kwargs must not contain the key "max_leaf_nodes"'
                ' (it will be set to n_bins automatically).'
            )

        if verbose:
            if tqdm is None:
                raise ImportError('If verbose is True, tqdm must be installed')
            tqdm_ = tqdm
        else:
            tqdm_ = lambda x: x  # noqa: E731

        if X.device.type != 'cpu' or y.device.type != 'cpu':
            warnings.warn(
                'Computing tree-based bins involves the conversion of the input PyTorch'
                ' tensors to NumPy arrays. The provided PyTorch tensors are not'
                ' located on CPU, so the conversion has some overhead.',
                UserWarning,
            )
        X_numpy = X.cpu().numpy()
        y_numpy = y.cpu().numpy()
        bins = []
        for column in tqdm_(X_numpy.T):
            feature_bin_edges = [float(column.min()), float(column.max())]
            tree = (
                (
                    sklearn_tree.DecisionTreeRegressor
                    if regression
                    else sklearn_tree.DecisionTreeClassifier
                )(max_leaf_nodes=n_bins, **tree_kwargs)
                .fit(column.reshape(-1, 1), y_numpy)
                .tree_
            )
            for node_id in range(tree.node_count):
                # The following condition is True only for split nodes. Source:
                # https://scikit-learn.org/1.0/auto_examples/tree/plot_unveil_tree_structure.html#tree-structure
                if tree.children_left[node_id] != tree.children_right[node_id]:
                    feature_bin_edges.append(float(tree.threshold[node_id]))
            bins.append(torch.as_tensor(feature_bin_edges).unique())
        _check_bins(bins)
        return [x.to(device=X.device, dtype=X.dtype) for x in bins]

def emb_ple(x, bin):
    x = np.array(x)
    x_emb = np.zeros(len(bin)-1)
    for i in range(len(x_emb)):
        if (x < bin[i]) and (i > 0):
            x_emb[i] = 0
        elif  (x >= bin[i+1]) and (i < len(x_emb)):
            x_emb[i] = 1
        else:
             x_emb[i] = (x - bin[i])/(bin[i+1] - bin[i])
    return x_emb


def ple(bins, x):
    x = np.array(x)
    bin = bins[0].numpy() # for torch tensors
    #bin = bins[0]
    f1 = emb_ple(x[0, 0], bin)
    for k in range(len(x)-1):
        f2 = emb_ple(x[k+1, 0], bin)
        f1 = np.vstack([f1, f2])
    x_ple = f1
    for i in range(len(x[0])-1):
        bin = bins[i+1].numpy()
        #bin = bins[i+1]
        f1 = emb_ple(x[0, i+1], bin)
        for k in range(len(x)-1):
            f2 = emb_ple(x[k+1, i+1], bin)
            f1 = np.vstack([f1, f2])
        x_ple = np.concatenate([x_ple, f1], axis = 1)
    return x_ple

# x = np.array([[5, 1, 6, 3], [2, 5, 1, 3], [5, 1, 6, 3]])
# bin = np.array([[0, 2, 4, 8, 10], [0, 2, 4, 8, 10], [0, 2, 4, 8, 10], [0, 2, 4, 8, 10]])
# bins = compute_bins(torch.from_numpy(x_train))
# x_emb = ple(bins, x_train)
# print(x_emb)

Тут я загружаю данные и разделяю их.  
Для некоторых моделей следует использовать шкалирование, а на такие, как catboost шкалирование не влияет. Поэтому перед разделением проведено шкалирование.

In [8]:
# housing = fetch_california_housing()
# x = housing.data
# y = housing.target
# scaler = StandardScaler()
# scaler.fit_transform(x)
def archive(file):
  archive = rarfile.RarFile(file)
  archive.extractall()
  archive.close()
  return archive
archive('gesture.rar')
x = pd.DataFrame(data=np.concatenate((np.load('gesture/X_num_train.npy'), np.load('gesture/X_num_val.npy'), np.load('gesture/X_num_test.npy')), axis=0))
y = pd.Series(data=np.concatenate((np.load('gesture/y_train.npy'), np.load('gesture/y_val.npy'), np.load('gesture/y_test.npy')), axis=0))
categorical_features = x.select_dtypes(include=['category', 'object']).columns
for feature in categorical_features:
  x[feature] = x[feature].astype(str)
numeric_categorical_indices = [x.columns.get_loc(col) for col in categorical_features]
numeric_features = x.select_dtypes(include=['number'])
if not numeric_features.empty:
  scaler = StandardScaler()
  x[numeric_features.columns] = scaler.fit_transform(numeric_features)

label_encoder = LabelEncoder()
for feature in categorical_features:
  x[feature] = label_encoder.fit_transform(x[feature])
if pd.api.types.is_categorical_dtype(pd.Series(y)):
  y = y.astype(str)
  y = label_encoder.fit_transform(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)
#x_train_val, x_test, y_train_val, y_test = train_test_split(x, y)
#x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val)

[[           nan  1.0000000e+00            nan ...  5.7159525e-40
   1.9208524e-01  5.7159525e-40]
 [           nan  1.0000000e+00            nan ...  2.8579762e-40
   9.6492887e-02  2.8579762e-40]
 [           nan  1.0000000e+00            nan ...  0.0000000e+00
  -0.0000000e+00  0.0000000e+00]
 ...
 [           nan  1.0000000e+00            nan ...  0.0000000e+00
  -0.0000000e+00  0.0000000e+00]
 [           nan  1.0000000e+00            nan ...  5.7159525e-40
   1.9208524e-01  5.7159525e-40]
 [           nan  1.0000000e+00            nan ...  5.7159525e-40
   1.9208524e-01  5.7159525e-40]]


### Стандартные модели

Обучаем модели из коробки

Обучение на всех моделях кроме LightGBM. Её обучаю отдельно из-за того, что она нестантартно запускается.

In [ ]:
df, models = project_train_simple(x_train, x_test, y_train, y_test)

In [ ]:
df

Обучение LGBM

In [ ]:
# train_dataset = lgb.Dataset(x_train, y_train)
# test_dataset = lgb.Dataset(x_test, y_test)

# booster = lgb.train({"objective": "regression"},
#                     train_set=train_dataset, valid_sets=(test_dataset,),
#                     num_boost_round=20)
# df.loc[ len(df.index )] = ['LGBM',
#                            (accuracy_score(y_train, booster.predict(x_train))),
#                            accuracy_score(y_test, booster.predict(x_test))]

Статистика по моделям из коробки по сравнению с значением метрики в статье.

In [ ]:
p_values = []
for i, model in enumerate(models):
    rsme = np.array(boostraping(model, x_test, y_test))
    print(rsme.mean())
    p_value = one_sample_t_test(rsme, 0.683, tail="two")
    p_values.append(p_value)
# rsme = boostraping(booster, x_test, y_test)
# p_value = one_sample_t_test(rsme, 0.683, tail="two")
# p_values.append(p_value)
df['p_value'] = p_values

In [ ]:
df

In [ ]:
# @title model name

from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('model name').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

Заметно, что catboost из коробки дает результат практически равный полученному в статье(больше на 0.013), чем в статье.  
При этом хуже всего показал себя RandomForestRegressor - результат более чем вдвое хуже получененого в статье(0.85 против 0.43).  
Попробуем улучшить результат результат в следующем разделе. При этом все результаты статистически отличаются от полученного в статье

### Модели после GridsearchCV

Проведем обучение с использованием уже написанных функций.

In [ ]:
df2, best_params = project_train(x_train, x_test, y_train, y_test)


Результат выводится в виде датафрейма для простоты работы с ним.

In [ ]:
df2

Отдельно обучим LGBM

In [ ]:
# param_grid = {
#     'num_leaves': [31, 127],
#     'reg_alpha': [0.1, 0.5],
#     'max_depth': [-1, 2, 5],
#     'n_estimators': [100, 300],
#     'n_jobs': [-1]
#     }
# lgb_estimator = lgb.LGBMRegressor()
# gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid)
# booster = gsearch.fit(X=x_train, y=y_train)
# #booster = lgb.train({"objective": "regression"},
# #                    train_set=train_dataset, valid_sets=(test_dataset,),
# #                    num_boost_round=20)
# df2.loc[ len(df2.index )] = ['LGBM',
#                            (mean_squared_error(y_train, booster.predict(x_train), squared = False)),
#                            mean_squared_error(y_test, booster.predict(x_test), squared = False)]

Проверим получилось ли статистически значимо улучшить результат по сравнению по сравненимю с обучением на моделях из коробки.

In [ ]:
# best_params.append(booster)
#best_params = best_params[0:5]
p_values = []
for i, model in enumerate(best_params):
    rsme = np.array(boostraping(model, x_test, y_test))
    print(rsme.mean())
    p_value = one_sample_t_test(rsme, df['p_value'][i], tail="two")
    p_values.append(p_value)
df2['p_value'] = p_values
df2

Заметно, что удалось статистически значимо улучшить результаты для XGBRegressor, RandomForestRegressor и LGBM.

### Числовые эмбединги PLE-Q

Переведем данные с использованием эмбединга.

In [ ]:
bins = compute_bins(torch.from_numpy(x_train))#ple-q
x_train_emb = ple(bins, x_train)
x_test_emb = ple(bins, x_test)
'''
bins = compute_bins(
    torch.from_numpy(x_train),
    tree_kwargs={'min_samples_leaf': 64, 'min_impurity_decrease': 1e-4},
    y=torch.from_numpy(y_train),
    regression=True,
)
x_train_emb = ple(bins, x_train)
x_test_emb = ple(bins, x_test) #ple-t
'''

In [ ]:
df4, models_simple_ple_q = project_train_simple(x_train, x_test, y_train, y_test)
# train_dataset = lgb.Dataset(x_train, y_train)
# test_dataset = lgb.Dataset(x_test, y_test)

# booster = lgb.train({"objective": "regression"},
#                     train_set=train_dataset, valid_sets=(test_dataset,),
#                     num_boost_round=20)
# df4.loc[ len(df4.index )] = ['LGBM',
#                            (mean_squared_error(y_train, booster.predict(x_train), squared = False)),
#                            mean_squared_error(y_test, booster.predict(x_test), squared = False)]

In [ ]:
df4

Обучим модели

In [ ]:
df3, best_params_ple_q = project_train(x_train_emb, x_test_emb, y_train, y_test)
# param_grid = {
#     'num_leaves': [31, 127],
#     'reg_alpha': [0.1, 0.5],
#     'max_depth': [-1, 2, 5],
#     'n_estimators': [100, 300],
#     'n_jobs': [-1]
#     }
# lgb_estimator = lgb.LGBMRegressor()
# gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid)
# booster = gsearch.fit(X=x_train, y=y_train)
# #booster = lgb.train({"objective": "regression"},
# #                    train_set=train_dataset, valid_sets=(test_dataset,),
# #                    num_boost_round=20)
# df3.loc[ len(df3.index )] = ['LGBM',
#                            (mean_squared_error(y_train, booster.predict(x_train), squared = False)),
#                            mean_squared_error(y_test, booster.predict(x_test), squared = False)]

In [ ]:
df3

Проверим статистическую значимость различия использования эмбедингов на классическом ML

In [2]:
def onetail(grid_models, emb_models, x_test, x_test_emb, y_test, alpha = 0.05):
    for i in range(len(grid_models)):
        rsme_grid = np.array(boostraping(grid_models[i], x_test, y_test))
        rsme_emb = np.array(boostraping(emb_models[i], x_test_emb, y_test))
        res = stats.levene(rsme_grid, rsme_emb)
        if (res.pvalue > alpha):
          t_test = stats.ttest_ind(rsme_grid, rsme_emb)
        else:
          t_test = stats.ttest_ind(rsme_grid, rsme_emb, equal_var = False)
        p_value = t_test.pvalue
        # Step 5: Interpret the p-value
        print("P-value:", p_value)

        if (p_value*0.5 < alpha) and (t_test.statistic<0):
          print(
            "Reject the null hypothesis. There is a significant stat improvement for the model "
            +str(grid_models[i].__class__.__name__)+'.')
        else:
          print("Fail to reject the null hypothesis. There is no statistically significant difference for model "
          +str(grid_models[i].__class__.__name__)+'.')

## House_16H


Этот датасет пришлось загружать, поэтому он взят с локального устройства.(эта часть проекта сделана в VSC)

### Модели из коробки

Загрузили, отшкалировали. Теперь обучаем из коробки.

In [10]:
best_params = best_params[0:4]
onetail(best_params, best_params_ple_q, x_test, x_test_emb, y_test)

NameError: name 'best_params' is not defined

### Числовые эмбединги PLE-T

In [ ]:
bins = compute_bins(
    torch.from_numpy(x_train),
    tree_kwargs={'min_samples_leaf': 64, 'min_impurity_decrease': 1e-4},
    y=torch.from_numpy(y_train),
    regression=True,
)
x_train_emb = ple(bins, x_train)
x_test_emb = ple(bins, x_test)

In [ ]:
df3, best_params_ple_q = project_train(x_train_emb, x_test_emb, y_train, y_test)

In [ ]:
onetail(best_params, best_params_ple_q, x_test, x_test_emb, y_test)

### Pereodic


In [11]:
from torch.nn.parameter import Parameter

def periodic_embedding(X, n_features):
  x = torch.from_numpy(np.array(X))
  k = 48
  sigma = 0.01
  weight = Parameter(torch.empty(n_features, k))
  x = 2 * math.pi * weight * x[..., None]
  x = torch.cat([torch.cos(x), torch.sin(x)], -1)
  x = x.detach().numpy()
  x_f = x[:, 0, :]
  for i in range(x.shape[1]-1):
    x_f = np.concatenate([x_f, x[:, i+1, :]], axis = 1)

  return x_f

In [12]:
x_train.shape[1]

1056

In [13]:
x = periodic_embedding(x_train, x_train.shape[1])

In [14]:
x.shape

(7500, 101376)

In [15]:
x_train.shape

(7500, 1056)

In [ ]:
x_train_emb = periodic_embedding(x_train, x_train.shape[1])
x_test_emb = periodic_embedding(x_test, x_test.shape[1])

In [ ]:
df3, best_params_ple_q = project_train(x_train_emb, x_test_emb, y_train, y_test)

In [ ]:
df2, best_params = project_train(x_train, x_test, y_train, y_test)

In [3]:
onetail(best_params, best_params_ple_q, x_test, x_test_emb, y_test)

NameError: name 'best_params' is not defined